In [1]:
import requests
import pandas as pd
import io

In [2]:
url = 'https://api.bluelytics.com.ar/v2/evolution.csv'

# Descarga el contenido del CSV
response = requests.get(url)
if response.status_code == 200:
    data = response.text
else:
    print("Error al descargar el CSV:", response.status_code)
    exit()

# Procesa el CSV con pandas
df = pd.read_csv(io.StringIO(data))

# Ahora puedes trabajar con el DataFrame 'df'
print(df.head())  # Muestra las primeras filas del DataFrame


          day     type  value_buy  value_sell
0  2024-05-10  Oficial      866.0       924.0
1  2024-05-10     Blue     1010.0      1040.0
2  2024-05-09  Oficial      866.0       924.0
3  2024-05-09     Blue     1015.0      1045.0
4  2024-05-08  Oficial      866.0       924.0


In [21]:
import psycopg2
import os

try:
    # Replace placeholders with your actual credentials
    host = 'localhost'
    port = 5432  # Default PostgreSQL port
    database = 'staging'
    user = 'admin'
    password = 'admin' 

    connection = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
        )

    cursor = connection.cursor()

    ##############################################3


    # Schema (assuming "staging" is a schema, not a database)
    table_name = 'daily_dolar'

    # Check if table exists 
    check_table_exists = f"""
        SELECT EXISTS (
            SELECT 1
            FROM information_schema.tables
            WHERE table_name = '{table_name}'
        );
    """
    cursor.execute(check_table_exists)
    exists = cursor.fetchone()[0]  # Get the existence check result

    if not exists:  # Create table only if it doesn't exist
        create_table = f"""    
        CREATE TABLE {table_name} (
            day DATE,     
            type VARCHAR(255) NOT NULL,  
            value_buy DECIMAL(10,2) NOT NULL,  
            value_sell DECIMAL(10,2) NOT NULL,
            PRIMARY KEY (day, type)
        );
        """
        cursor.execute(create_table)
        connection.commit()
        print(f"Table '{table_name}' created successfully!")

    # Assuming your DataFrame (df) has columns matching the table schema
    # Prepare data for insertion using list comprehension
    data_tuples = [
        (row['day'], row['type'], row['value_buy'], row['value_sell'])
        for index, row in df.iterrows()
    ]

    # Insert data in batches (optional for efficiency with large datasets)
    insert_query = """
        INSERT INTO daily_dolar (day, type, value_buy, value_sell)
        VALUES (%s, %s, %s, %s)
    """
    cursor.executemany(insert_query, data_tuples)
    connection.commit()
    ################################################
 
    print("Data inserted successfully!")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)

Table 'daily_dolar' created successfully!
Data inserted successfully!


In [ ]:
import psycopg2
import os
from airflow.operators.python_operator import PythonOperator
from airflow.hooks.base_hook import BaseHook


try:
    # Replace placeholders with your actual credentials
    staging_conn = BaseHook.get_connection("staging")

    connection = psycopg2.connect(
        host=staging_conn.host,
        port=staging_conn.port,
        database=staging_conn.database,
        user=staging_conn.user,
        password=staging_conn.password
        )

    cursor = connection.cursor()

    ##############################################3


    # Schema (assuming "staging" is a schema, not a database)
    table_name = 'daily_dolar'

    # Check if table exists 
    check_table_exists = f"""
        SELECT EXISTS (
            SELECT 1
            FROM information_schema.tables
            WHERE table_name = '{table_name}'
        );
    """
    cursor.execute(check_table_exists)
    exists = cursor.fetchone()[0]  # Get the existence check result

    if not exists:  # Create table only if it doesn't exist
        create_table = f"""    
        CREATE TABLE {table_name} (
            day DATE,     
            type VARCHAR(255) NOT NULL,  
            value_buy DECIMAL(10,2) NOT NULL,  
            value_sell DECIMAL(10,2) NOT NULL,
            PRIMARY KEY (day, type)
        );
        """
        cursor.execute(create_table)
        connection.commit()
        print(f"Table '{table_name}' created successfully!")

    # Assuming your DataFrame (df) has columns matching the table schema
    # Prepare data for insertion using list comprehension
    data_tuples = [
        (row['day'], row['type'], row['value_buy'], row['value_sell'])
        for index, row in df.iterrows()
    ]

    # Insert data in batches (optional for efficiency with large datasets)
    insert_query = """
        INSERT INTO daily_dolar (day, type, value_buy, value_sell)
        VALUES (%s, %s, %s, %s)
    """
    cursor.executemany(insert_query, data_tuples)
    connection.commit()
    ################################################
 
    print("Data inserted successfully!")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)